# Intro

This notebook demonstrates the use of the tweeteater functions. The goal of the tweeteater library is to provide functions that allow simple loading of large Twitter datasets collected from the streaming API. It particularly allows to extract nested observations of engagements.

# Parameters

In [1]:
# the directory with the tweets 
directory = 'sample_data'
# the extension of the tweet files in the directory
extension = '.jsonl'

# Load, subset tweets and analyse types

At first, we now want to load all tweets in the directory and filter them. We can filter on two conditions, 1. being that we only keep certain types of tweet, 2. we can provide a custom subsetting function. Regarding 1.: a tweet can be a a) retweet of a tweet, b) a replie to a tweet c) a quote of a tweet or d) just an original tweet, that is a tweet that is neither a) - c). A) - c) are not necessarily mutually exclusive.  

In [3]:
from lib.eat_tweets import *
from lib.utils import *

We now only want to keep tweets made by certain users, which are in this case political accounts, and hence define a custom subset function that acts directly on the tweet object 

In [4]:
pt = directory + '/screenname2party.json'
politicians_screennames = set([scn for scn, party in load_json(pt).items()])
subset_func = lambda tweet: safe_get(tweet, *('user', 'screen_name')) in politicians_screennames

Furthermore, we decide that we want to keep all tweettypes.

In [5]:
keep_tweettypes = ['original', 'retweet', 'reply', 'quote']

Now we are ready to load the tweets.

In [6]:
filepaths = get_filepaths(directory, extension)
n_filepaths = len(list(get_filepaths(directory, extension)))
tweettypes = eat_tweettypes(filepaths, subset_func=subset_func, n_filepaths=n_filepaths)

  0%|          | 0/24 [00:00<?, ?it/s]

The tweettypes object is a generator using lazy loading. Since the sample dataset used here is small, we can safely convert it into a dictionary, but with larger datasets, this should be avoided. 

In [7]:
tweettypes = dict(tweettypes); print('n tweets:', len(tweettypes))

100%|██████████| 24/24 [00:05<00:00,  6.63it/s]

n tweets: 578


We now want to see the distribution across tweettypes

In [9]:
import seaborn as sns 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

tweettypes_df = pd.DataFrame({'tweet type': [str(x) for x in tweettypes.values()]})
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
sns.countplot(data=tweettypes_df, y='tweet type', ax=ax)

Since we are only interested in original tweets, we subset accordingly

In [10]:
ids = [id_ for id_, types in tweettypes.items() if 'original' in types]
print('n original tweets', len(ids))

n original tweets 113


# Extract more attributes

We want to know a little more about the tweets, so we want to extract some more attributes. A comprehensive overview of all atributes of the tweet object can be found here: https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html

In [11]:
attributes = ['created_at', 'user.screen_name', 'user.name', 'text', 'lang']
filepaths = get_filepaths(directory, extension)
n_filepaths = len(list(get_filepaths(directory, extension)))
tweet_attributes = eat_tweetattributes(filepaths, ids, attributes, n_filepaths=n_filepaths)
tweet_attributes = pd.DataFrame([[a] + b for a, b in tweet_attributes])
tweet_attributes.columns = ['id'] + attributes

100%|██████████| 24/24 [00:03<00:00,  6.97it/s]


In [12]:
tweet_attributes.head()

,id,created_at,user.screen_name,user.name,text,lang
0,899636290458718208,Mon Aug 21 14:16:20 +0000 2017,CDU,CDU Deutschlands,.@OttosKassiber in der @zeitonline zum 2%-Ziel...,de
1,899639483334893569,Mon Aug 21 14:29:01 +0000 2017,rennersen,Tim Renner,Endlich Sonnyboy #timrennt #BTW17 #Charlottenb...,en
2,899640397542117376,Mon Aug 21 14:32:39 +0000 2017,PaulZiemiak,Paul Ziemiak,Tür zu Tür Auftakt zusammen mit @MaikBeermann....,de
3,899640438180773888,Mon Aug 21 14:32:49 +0000 2017,JensBitzka,Jens Bitzka,Kandidatencheck - 4 Kandidierende aus dem Wahl...,de
4,899641325330591744,Mon Aug 21 14:36:21 +0000 2017,babman,M. Babenhauserheide,Mein kleiner Beitrag zum WDR-Kandidatencheck. ...,de


Let's also save this

In [14]:
tweet_attributes.to_csv('sample_output/tweets.csv')

# Extracting engagements

We now also want to know, how many engagements (favourites, replies, retweets, quotes) the original tweets from the politicians received. We can extract this in the following way

In [22]:
filepaths = get_filepaths(directory, extension)
n_filepaths = len(list(get_filepaths(directory, extension)))
engagements = eat_engagements(filepaths, ids, n_filepaths=n_filepaths)
engagements = pd.DataFrame(list(engagements))
engagements.columns = ['observed timestamp', 'observed in tweet.id', 'tweet.id', 
                       'engagement type', 'count']

100%|██████████| 24/24 [00:03<00:00,  6.41it/s]


In [23]:
engagements.head()

,observed timestamp,observed in tweet.id,tweet.id,engagement type,count
0,Mon Aug 21 14:04:06 +0000 2017,899633210744864770,899632451445825536,favourite_count,1
1,Mon Aug 21 14:04:06 +0000 2017,899633210744864770,899632451445825536,retweet_count,2
2,Mon Aug 21 14:04:28 +0000 2017,899633304693026818,899632451445825536,favourite_count,1
3,Mon Aug 21 14:04:28 +0000 2017,899633304693026818,899632451445825536,retweet_count,3
4,Mon Aug 21 14:04:30 +0000 2017,899633310867103746,899615666386415617,favourite_count,18


Note that the engagement types 'favourite_count' and 'retweet_count' correspond to the actual counts according to Twitter at the time 'tweet extracted from' was made. Hence, these are cumulative counts. 'reply' and 'quote' simply mark a reply or a quote within our dataset, hence the count is always 1. Let's now see, which tweets have the highest engagements

Let's now add some more columns to this. 

In [25]:
tweetid2screenname = dict(zip(tweet_attributes['id'], tweet_attributes['user.screen_name']))
tweetid2createdat = dict(zip(tweet_attributes['id'], tweet_attributes['created_at']))
engagements['user.screen_name'] = [tweetid2screenname[id_]
                                   for id_ in engagements['tweet.id']]
engagements['created_at'] = [tweetid2createdat[id_]
                             for id_ in engagements['tweet.id']]

In [26]:
engagements.head()

,observed timestamp,observed in tweet.id,tweet.id,engagement type,count,user.screen_name,created_at
0,Mon Aug 21 14:04:06 +0000 2017,899633210744864770,899632451445825536,favourite_count,1,DieLinkeNRW,Mon Aug 21 14:01:05 +0000 2017
1,Mon Aug 21 14:04:06 +0000 2017,899633210744864770,899632451445825536,retweet_count,2,DieLinkeNRW,Mon Aug 21 14:01:05 +0000 2017
2,Mon Aug 21 14:04:28 +0000 2017,899633304693026818,899632451445825536,favourite_count,1,DieLinkeNRW,Mon Aug 21 14:01:05 +0000 2017
3,Mon Aug 21 14:04:28 +0000 2017,899633304693026818,899632451445825536,retweet_count,3,DieLinkeNRW,Mon Aug 21 14:01:05 +0000 2017
4,Mon Aug 21 14:04:30 +0000 2017,899633310867103746,899615666386415617,favourite_count,18,Die_Gruenen,Mon Aug 21 12:54:23 +0000 2017


In [27]:
engagements.to_csv('sample_output/engagements.csv')